In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Load data
df = pd.read_csv('airline-passengers.csv')
timeseries = df[["Passengers"]].values.astype('float32')

# Train-test split
train_size = int(len(timeseries) * 0.67)
test_size = len(timeseries) - train_size
train, test = timeseries[:train_size], timeseries[train_size:]

def create_dataset(dataset, lookback):
    X, y = [], []
    for i in range(len(dataset)-lookback):
        features = dataset[i:i+lookback]
        features = np.array([float(array[0]) for array in features])
        target = dataset[i+lookback:i+lookback+1][0][0]
        X.append(features)
        y.append(target)      
    return torch.tensor(np.array(X).astype(np.float32)), torch.tensor(np.array(y))

lookback = 14
X_train, y_train = create_dataset(train, lookback=lookback)
X_test, y_test = create_dataset(test, lookback=lookback)

train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=5, shuffle=False)

test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=5, shuffle=False)

class LSTMModel(nn.Module):
    def __init__(self, hidden_size, seed):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size=1, hidden_size=hidden_size, num_layers=1, batch_first=True)
        self.linear = nn.Linear(hidden_size * lookback, 1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.3)
    
    def forward(self, x):
        x = x.unsqueeze(-1)
        batch_size = x.size(0)
        h_0 = torch.zeros(1, batch_size, self.hidden_size)
        c_0 = torch.zeros(1, batch_size, self.hidden_size)
        x, (hn, cn) = self.lstm(x, (h_0, c_0))
        x = x.reshape(batch_size, -1)
        x = self.dropout(x)
        x = self.linear(x)
        return x, hn, cn

class ANNModel(nn.Module):
    def __init__(self, hidden_size):
        super(ANNModel, self).__init__()
        self.linear1 = nn.Linear(hidden_size, int(hidden_size / 2))
        self.linear2 = nn.Linear(int(hidden_size / 2), 1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.3)
    
    def forward(self, x):
        x = self.relu(self.linear1(x))
        x = self.dropout(x)
        x = self.relu(self.linear2(x))
        return x

class RESmodel(nn.Module):
    def __init__(self):
        super(RESmodel, self).__init__()
        self.linear1 = nn.Linear(4, 30)
        self.linear2 = nn.Linear(30, 1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.3)
    
    def forward(self, x):
        x = self.relu(self.linear1(x))
        x = self.dropout(x)
        x = self.relu(self.linear2(x))
        return x
    
class PredModel(nn.Module):
    def __init__(self, inputsize):
        self.inputsize = inputsize
        super(PredModel, self).__init__()
        self.linear1 = nn.Linear(inputsize, inputsize*3)
        self.linear2 = nn.Linear(inputsize*3, inputsize)
        self.linear3 = nn.Linear(inputsize, 1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.3)
    
    def forward(self, x):
        x = self.relu(self.linear1(x))
        x = self.dropout(x)
        x = self.relu(self.linear2(x))
        x = self.dropout(x)
        x = self.relu(self.linear3(x))
        return x

def train_lstm(model, criterion, optimizer):
    model.train()
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        y_pred, _, _ = model(X_batch.float())
        loss = criterion(y_pred.squeeze(), y_batch)
        loss.backward()
        optimizer.step()
    return model

def train_ann(model, criterion, optimizer, state, y):
    model.train()
    optimizer.zero_grad()
    y_pred = model(state)
    loss = criterion(y_pred.squeeze(), y)
    loss.backward()
    optimizer.step()
    return model

def train_res(pred_model, hn_ver_model, cn_ver_model, res_model, pred_ann_model, criterion, optimizer):
    res_model.train()
    for X_batch, y_batch in train_loader:
        with torch.no_grad():
            pred, hn, cn = pred_model(X_batch)
            h_pred = hn_ver_model(hn)
            c_pred = cn_ver_model(cn)
            pa_pred = pred_ann_model(X_batch)
            X_joined = torch.stack([pred.view(pred.size(0)), h_pred.view(h_pred.size(1)), c_pred.view(c_pred.size(1)), pa_pred.view(pred.size(0))], 1)
        optimizer.zero_grad()
        y_pred = res_model(X_joined)
        loss = criterion(y_pred.squeeze(), y_batch)
        loss.backward()
        optimizer.step()
    return res_model

def train_ann_pred(model, criterion, optimizer):
    model.train()
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        y_pred = model(X_batch)
        loss = criterion(y_pred.squeeze(), y_batch)
        loss.backward()
        optimizer.step()
    return model

def predict(pred_model, hn_ver_model, cn_ver_model, res_model, pred_ann_model, X):
    with torch.no_grad():
        pred, hn, cn = pred_model(X)
        h_pred = hn_ver_model(hn)
        c_pred = cn_ver_model(cn)
        pa_pred = pred_ann_model(X)
        X_joined = torch.stack([pred.view(pred.size(0)), h_pred.view(h_pred.size(1)), c_pred.view(c_pred.size(1)), pa_pred.view(pred.size(0))], 1)
        return res_model(X_joined)

def objective(params):
    hidden_size = params["hidden_size"]
    seed = params["seed"]
    train_epochs = 100
    
    pred_lstm_model = LSTMModel(hidden_size, seed)
    hn_ver_model = ANNModel(hidden_size)
    cn_ver_model = ANNModel(hidden_size)
    ver_model = RESmodel()
    pred_ann_model = PredModel(lookback)

    for _ in range(10):
        criterion = nn.MSELoss(reduction='mean')
        
        optimizer_lstm = torch.optim.Adam(pred_lstm_model.parameters(), lr=0.0015, weight_decay=0.0003)
        for _ in range(train_epochs):
            pred_lstm_model = train_lstm(pred_lstm_model, criterion, optimizer_lstm)

        optimizer_hn = torch.optim.Adam(hn_ver_model.parameters(), lr=0.0015, weight_decay=0.0003)
        for _ in range(train_epochs):
            for X_batch, y_batch in train_loader:
                _, hn, _ = pred_lstm_model(X_batch)
                hn_ver_model = train_ann(hn_ver_model, criterion, optimizer_hn, hn, y_batch)

        optimizer_cn = torch.optim.Adam(cn_ver_model.parameters(), lr=0.0015, weight_decay=0.0003)
        for _ in range(train_epochs):
            for X_batch, y_batch in train_loader:
                _, _, cn = pred_lstm_model(X_batch)
                cn_ver_model = train_ann(cn_ver_model, criterion, optimizer_cn, cn, y_batch)

        optimizer_pred_ann = torch.optim.Adam(pred_ann_model.parameters(), lr=0.0015, weight_decay=0.0003)
        for _ in range(train_epochs):
            pred_ann_model = train_ann_pred(pred_ann_model, criterion, optimizer_pred_ann)

        optimizer_ver = torch.optim.Adam(ver_model.parameters(), lr=0.0015, weight_decay=0.0003)
        for _ in range(train_epochs):
            ver_model = train_res(pred_lstm_model, hn_ver_model, cn_ver_model, ver_model, pred_ann_model, criterion, optimizer_ver)

        pred_lstm_model.eval()
        hn_ver_model.eval()
        cn_ver_model.eval()
        ver_model.eval()
        pred_ann_model.eval()

        with torch.no_grad():
            train_y_pred = predict(pred_lstm_model, hn_ver_model, cn_ver_model, ver_model, pred_ann_model, X_train)
            test_y_pred = predict(pred_lstm_model, hn_ver_model, cn_ver_model, ver_model, pred_ann_model, X_test)

            y_pred = train_y_pred.squeeze().numpy()
            y_true = y_train.numpy()
            mse = mean_squared_error(y_true, y_pred)
            mae = mean_absolute_error(y_true, y_pred)
            r2 = r2_score(y_true, y_pred)
            print("Scores")
            print(f"Train: Mean Squared Error (MSE): {mse:.2f}, Mean Absolute Error (MAE): {mae:.2f}, R-squared (R2): {r2:.2f}")

            y_pred = test_y_pred.squeeze().numpy()
            y_true = y_test.numpy()
            mse = mean_squared_error(y_true, y_pred)
            mae = mean_absolute_error(y_true, y_pred)
            r2 = r2_score(y_true, y_pred)
            print(f"Test: Mean Squared Error (MSE): {mse:.2f}, Mean Absolute Error (MAE): {mae:.2f}, R-squared (R2): {r2:.2f}")

    return train_y_pred.squeeze().cpu().numpy(), test_y_pred.squeeze().cpu().numpy()

test_params = {
    "hidden_size": 50,
    "seed": 5
}

y1, y2 = objective(test_params)
arr = np.concatenate((y1, y2))

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(8, 5))  # Adjust figure size if needed
x_values = range(len(arr))
plt.plot(x_values, arr, marker='o', linestyle='-', color='r', label='Float Values')
true_values = np.concatenate((y_train.numpy(), y_test.numpy()))
plt.plot(range(len(true_values)), true_values, marker='x', linestyle='-', color='g', label='True Values')

plt.xlabel('Index')
plt.ylabel('Value')
plt.title('Visualization of List of Float Values')
plt.legend()
plt.grid(True)
plt.show()